In [23]:
%pip install -U openai-whisper

In [24]:
%pip install yt-dlp

Note: you may need to restart the kernel to use updated packages.


In [25]:
import yt_dlp

def descargar_audio_youtube(url, nombre_salida="audio"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': nombre_salida,
        'postprocessors': [{  # Convertir a mp3
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
        'no_warnings': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

if __name__ == "__main__":
    url_video = "https://youtu.be/PlGQtL0oI1k?si=81I-gYAL2YKO-h6t"
    descargar_audio_youtube(url_video)
    print("¡Audio descargado y convertido a mp3!")


¡Audio descargado y convertido a mp3!                    


In [26]:
from transformers import pipeline

def transcribir_audio_hf(ruta_audio: str) -> str:
    # Cargar pipeline de transcripción Whisper en español
    transcriptor = pipeline("automatic-speech-recognition", model="openai/whisper-tiny", device=-1, chunk_length_s=30)

    # Transcribir el audio
    resultado = transcriptor(ruta_audio)
    
    # Retornar texto transcrito
    return resultado['text']

if __name__ == "__main__":
    archivo_audio = "C:/Users/JL/Downloads/X/audio.mp3"  # Cambia aquí por tu archivo
    texto = transcribir_audio_hf(archivo_audio)
    print("Transcripción:\n", texto)


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


Transcripción:
  Bueno, esta recermana, tengo una pregunta que hacerle. En Mateo 8 del verso 28 al 34, pero sólo le voy a hacer la pregunta. Sí, sí, sí. Bueno, en Mateo 8, 28 al 34. Sí. Después de haber libertado de espíritus a los en demoniados, el pueblo le pidió injustamente al señor Jesucristo que se fueran. Sería que no quería que el libertara a más almas o porque no creían en él. Bueno, porque esa región era una región de gente increíble y estaban allí escuchan y ellos vivían de burujerías, de chicerías, vivían adorando demonios, adorando espíritus, idolos, y por supuesto que ellos también tenían sus adivinos, sus magos, sus hechiceros, y cuando el Señor Jesucristo apareció ahí, predicándole van Evangelio de Asiéndolos milagros y dice que el señor libertó a esas dos personas eran dos personas, dice que estaban con demonios que tenían muchos demonios cuando esas personas quedaron libertadas de esos demonios ellos dijeron, no nos conviene, no nos conviene porque este nos vino a qui

In [27]:
# from transformers import pipeline

# def transcribir_audio_hf(ruta_audio: str) -> str:
#     # Cargar pipeline de transcripción Whisper en español
#     transcriptor = pipeline("automatic-speech-recognition", model="openai/whisper-tiny", device=-1, chunk_length_s=30)

#     # Transcribir el audio
#     resultado = transcriptor(ruta_audio)
    
#     # Retornar texto transcrito
#     return resultado['text']

# if __name__ == "__main__":
#     archivo_audio = "C:/Users/JL/Downloads/QueridaYo.mp3"  # Cambia aquí por tu archivo
#     texto = transcribir_audio_hf(archivo_audio)
#     print("Transcripción:\n", texto)


In [34]:
# HF_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
import os

In [35]:
# %pip install sentence-transformers faiss-cpu transformers accelerate huggingface-hub
# %pip install langchain 
%pip install faiss-cpu
%pip install transformers sentence-transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [36]:
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from sentence_transformers import SentenceTransformer
import numpy as np

# ---------------------------
# CONFIGURACIÓN GLOBAL
# ---------------------------

# Forzamos CPU
device = torch.device("cpu")

In [31]:
# Cargar embeddings para búsqueda semántica
print("Cargando modelo de embeddings...")
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device='cpu')

# Cargar modelo QA en español
print("Cargando modelo QA...")
qa_model_name = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=tokenizer, device=-1)

# ---------------------------
# FUNCIONES DEL RAG
# ---------------------------

def crear_indice_faiss(textos):
    """Crea un índice FAISS a partir de una lista de textos."""
    embeddings = embedder.encode(textos, convert_to_tensor=False, normalize_embeddings=True)
    embeddings = np.array(embeddings).astype("float32")

    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index, textos

def buscar_contexto(query, index, textos, k=3):
    """Busca los k textos más relevantes para la query."""
    query_emb = embedder.encode([query], convert_to_tensor=False, normalize_embeddings=True)
    query_emb = np.array(query_emb).astype("float32")
    scores, idxs = index.search(query_emb, k)
    resultados = [textos[i] for i in idxs[0]]
    return resultados

def responder_pregunta(pregunta, contexto):
    """Usa el modelo QA para responder usando el contexto."""
    combined_context = " ".join(contexto)
    result = qa_pipeline({
        "context": combined_context,
        "question": pregunta
    })
    return result['answer']

Cargando modelo de embeddings...
Cargando modelo QA...


Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [37]:
# Carga de contexto
texto_transcrito = texto

# 2. Contexto adicional (puedes agregar documentos, PDFs, etc.)
documentos_extra = [
    " "]

# Unir todo en un corpus
corpus = [texto_transcrito] + documentos_extra

# 3. Crear índice FAISS
indice, textos = crear_indice_faiss(corpus)

In [38]:
# 4. Hacer una pregunta
# pregunta_usuario = "¿Cómo funciona el algoritmo de recomendación de YouTube para personalizar el contenido que se muestra a cada usuario?"
# pregunta_usuario = "¿Cuándo nació Youtube?"
# pregunta_usuario = "¿Cuándo nació Youtube?"
pregunta_usuario = "¿Porqué los magos no querían al señor Jesucristo?"
contexto_relevante = buscar_contexto(pregunta_usuario, indice, textos)
respuesta = responder_pregunta(pregunta_usuario, contexto_relevante)

print("\n📌 Pregunta:", pregunta_usuario)
print("📚 Contexto usado:", contexto_relevante)
print("💡 Respuesta:", respuesta)


📌 Pregunta: ¿Porqué los magos no querían al señor Jesucristo?
📚 Contexto usado: [' Bueno, esta recermana, tengo una pregunta que hacerle. En Mateo 8 del verso 28 al 34, pero sólo le voy a hacer la pregunta. Sí, sí, sí. Bueno, en Mateo 8, 28 al 34. Sí. Después de haber libertado de espíritus a los en demoniados, el pueblo le pidió injustamente al señor Jesucristo que se fueran. Sería que no quería que el libertara a más almas o porque no creían en él. Bueno, porque esa región era una región de gente increíble y estaban allí escuchan y ellos vivían de burujerías, de chicerías, vivían adorando demonios, adorando espíritus, idolos, y por supuesto que ellos también tenían sus adivinos, sus magos, sus hechiceros, y cuando el Señor Jesucristo apareció ahí, predicándole van Evangelio de Asiéndolos milagros y dice que el señor libertó a esas dos personas eran dos personas, dice que estaban con demonios que tenían muchos demonios cuando esas personas quedaron libertadas de esos demonios ellos d